
# Fine-tune the model with the Amazon Customer Reviews Dataset and a set of prompts

Taken from Data Science on AWS

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33229979648, available=26790584320, percent=19.4, used=6025555968, free=14381158400, active=5656969216, inactive=11864399872, buffers=2768896, cached=12820496384, shared=884736, slab=912265216)


In [3]:
%store -r model_checkpoint

In [4]:
try:
    model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(model_checkpoint)

bigscience/bloom-560m


In [6]:
%store -r dataset_templates_name

In [7]:
try:
    dataset_templates_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(dataset_templates_name)

amazon_us_reviews/Wireless_v1_00


In [9]:
%store -r prompt_template_name

In [10]:
try:
    prompt_template_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [11]:
print(prompt_template_name)

Given the review body return a categorical rating


# Create prompts for few-shot, one-shot, zero-shot inference on sample data

In [12]:
import pandas as pd
import csv
file = './data-tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz'

# Read the file
df = pd.read_csv(file, delimiter="\t", quoting=csv.QUOTE_NONE, compression="gzip")

df.isna().values.any()
df = df.dropna()
df = df.reset_index(drop=True)    

print("Shape of dataframe {}".format(df.shape))

# Convert Pandas dataframes into Datasets
import datasets
from datasets import Dataset

# Create Dataset objects (Arrow PyTables) from Pandas dataframes
dataset = Dataset.from_pandas(df)

# Apply prompt    
from promptsource.templates import DatasetTemplates
prompt_templates = DatasetTemplates(dataset_templates_name) 

for template in prompt_templates.templates.values():
    print(template.get_name())

prompt = prompt_templates[prompt_template_name]
print(prompt.answer_choices)    
print(prompt.__dict__)

dataset = dataset.select(range(100)).map(lambda row : {'prompt': prompt.apply(row)[0], 'label': prompt.apply(row)[1]})
print("Shape of dataset {}".format(dataset.shape))

Shape of dataframe (145427, 15)
Generate review headline based on review body
Generate review based on rating and category
Given the review headline return a categorical rating
Generate review headline based on rating
Given the review body return a categorical rating
1 ||| 2 ||| 3 ||| 4 ||| 5
{'answer_choices': '1 ||| 2 ||| 3 ||| 4 ||| 5', 'id': 'e6a1bbde-715d-4dad-9178-e2bcfaf5c646', 'jinja': "Given the following review:\n{{review_body}}\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- {{ answer_choices | join('\\n- ') }} \n|||\n{{answer_choices[star_rating-1]}}", 'metadata': <promptsource.templates.Template.Metadata object at 0x7fdadbbf73d0>, 'name': 'Given the review body return a categorical rating', 'reference': 'Given the review body, return a categorical rating. '}


  0%|          | 0/100 [00:00<?, ?ex/s]

Shape of dataset (100, 17)


In [13]:
prompt = dataset[0]['prompt']
label = dataset[0]['label']
print(prompt)
print(label)

Given the following review:
I keep buying madden every year hoping they get back to football. This years version is a little better than last years -- but that's not saying much.The game looks great. The only thing wrong with the animation, is the way the players are always tripping on each other.<br /><br />The gameplay is still slowed down by the bloated pre-play controls. What used to take two buttons is now a giant PITA to get done before an opponent snaps the ball or the play clock runs out.<br /><br />The turbo button is back, but the player movement is still slow and awkward. If you liked last years version, I'm guessing you'll like this too. I haven't had a chance to play anything other than training and a few online games, so I'm crossing my fingers and hoping the rest is better.<br /><br />The one thing I can recommend is NOT TO BUY THE MADDEN BUNDLE. The game comes as a download. So if you hate it, there's no trading it in at Gamestop.
predict the associated rating from the 

# Perform zero-shot inference BEFORE fine-tuning

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [14]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [15]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [16]:
prompt0 = dataset[0]
prompt1 = dataset[1]
prompt2 = dataset[2]
prompt3 = dataset[3]

zero_shot_prompt = 'PROMPT: ' + prompt2['prompt'] + '\nRESPONSE:'

In [26]:
zero_shot_prompt

'PROMPT: Given the following review:\nIf you are prepping for the end of the world this is one of those things that you should have installed on your-end-of-the-world-proof PC.  Hail to the great Yuri!\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5\nRESPONSE:'

In [17]:
inputs = tokenizer(zero_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt2['label']))

PROMPT: Given the following review:
If you are prepping for the end of the world this is one of those things that you should have installed on your-end-of-the-world-proof PC.  Hail to the great Yuri!
predict the associated rating from the following choices (1 being lowest and 5 being highest)
- 1
- 2
- 3
- 4
- 5
RESPONSE: The
EXPECTED RESPONSE: 5


# Fine-tune the model with the Amazon Customer Reviews Data

In [18]:
from datasets import Dataset

lm_dataset_train = Dataset.from_parquet('./data/train/*.parquet')
print(lm_dataset_train.shape)

Using custom data configuration default-2d808a0fcba91d4c
Found cached dataset parquet (/root/.cache/huggingface/datasets/parquet/default-2d808a0fcba91d4c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


(30, 3)


In [19]:
from transformers import TrainingArguments
import torch

training_args = TrainingArguments(
    "{}-finetuned-amazon-customer-reviews".format(model_checkpoint.replace("/", "-")),
    learning_rate=2e-5,
    weight_decay=0.01, 
    max_steps=10,
    num_train_epochs=1.0,
    no_cuda=not torch.cuda.is_available()    
)

We pass along all of those to the `Trainer` class:

In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_train
)

max_steps is given, it will override any value given in num_train_epochs


In [21]:
train_results = trainer.train()
train_results

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 30
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10
  Number of trainable parameters = 559214592


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=10, training_loss=2.4359371185302736, metrics={'train_runtime': 220.0878, 'train_samples_per_second': 0.363, 'train_steps_per_second': 0.045, 'total_flos': 17645432733696.0, 'train_loss': 2.4359371185302736, 'epoch': 2.5})

# Save fine-tuned model

In [22]:
supervised_fine_tuned_model_path = './tmp_models/{}/'.format(model_checkpoint)

model.save_pretrained(supervised_fine_tuned_model_path)

Configuration saved in ./tmp_models/bigscience/bloom-560m/config.json
Configuration saved in ./tmp_models/bigscience/bloom-560m/generation_config.json
Model weights saved in ./tmp_models/bigscience/bloom-560m/pytorch_model.bin


In [23]:
%store supervised_fine_tuned_model_path

Stored 'supervised_fine_tuned_model_path' (str)


# Perform zero-shot inference AFTER fine-tuning

In [24]:
import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(supervised_fine_tuned_model_path)

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloom-560m/snapshots/e985a63cdc139290c5f700ff1929f0b5942cced2/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloom-560m/snapshots/e985a63cdc139290c5f700ff1929f0b5942cced2/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloom-560m/snapshots/e985a63cdc139290c5f700ff1929f0b5942cced2/tokenizer_config.json
loading configuration file ./tmp_models/bigscience/bloom-560m/config.json
Model config BloomConfig {
  "_name_or_path": "./tmp_models/bigscience/bloom-560m/",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropo

This model also supports many advanced parameters while performing inference including the following:

**max_length**: Model generates text until the output length (which includes the input context length) reaches max_length. If specified, it must be a positive integer.

**num_return_sequences**: Number of output sequences returned. If specified, it must be a positive integer.

**num_beams**: Number of beams used in the greedy search. If specified, it must be integer greater than or equal to num_return_sequences.

**no_repeat_ngram_size**: Model ensures that a sequence of words of no_repeat_ngram_size is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.

**temperature**: Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If temperature -> 0, it results in greedy decoding. If specified, it must be a positive float.

**early_stopping**: If True, text generation is finished when all beam hypotheses reach the end of stence token. If specified, it must be boolean.

**do_sample**: If True, sample the next word as per the likelyhood. If specified, it must be boolean.

**top_k**: In each step of text generation, sample from only the top_k most likely words. If specified, it must be a positive integer.

**top_p**: In each step of text generation, sample from the smallest possible set of words with cumulative probability top_p. If specified, it must be a float between 0 and 1.

**seed**: Fix the randomized state for reproducibility. If specified, it must be an integer.

In [25]:
inputs = tokenizer(zero_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt2['label']))

Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 3,
  "transformers_version": "4.26.1"
}



PROMPT: Given the following review:
If you are prepping for the end of the world this is one of those things that you should have installed on your-end-of-the-world-proof PC.  Hail to the great Yuri!
predict the associated rating from the following choices (1 being lowest and 5 being highest)
- 1
- 2
- 3
- 4
- 5
RESPONSE: 5
EXPECTED RESPONSE: 5
